<a href="https://colab.research.google.com/github/RC11-SkillsClass2022-23/22101933/blob/main/Detect_scene_for_vectorise_videos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

In [ ]:
import cv2
import os
import numpy as np
import shutil

In [ ]:
moving_root = '/content/drive/MyDrive/code_test/detect_scene/processed_videos/video_to_video'
static_root = '/content/drive/MyDrive/code_test/detect_scene/processed_videos/image_to_video'

In [ ]:
def VideoCategories(input_root):
  videos = os.listdir(input_root)
  threshold = 0.4
  for i in videos:
    prev_frame = None
    frame_count = 0
    scene_change_count = 0
    
    video_path = input_root + i
    video = cv2.VideoCapture(video_path)
    fps = video.get(cv2.CAP_PROP_FPS)
    
    while True:
      ret, frame = video.read()

      # Check if end of video
      if not ret:
          break

      # Convert the frame to grayscale
      gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

      # Resize the frame to a smaller size
      height, width = gray.shape
      scale_percent = 50
      new_width = int(width * scale_percent / 100)
      new_height = int(height * scale_percent / 100)
      resized = cv2.resize(gray, (new_width, new_height))

      # Calculate the histogram of the resized frame
      hist = cv2.calcHist([resized], [0], None, [256], [0, 256])

      # Normalize the histogram
      hist_norm = cv2.normalize(hist, hist)

      # Update the scene change counter if the histogram distance is greater than the threshold
      if prev_frame is not None:
          prev_hist = cv2.calcHist([prev_frame], [0], None, [256], [0, 256])
          prev_hist_norm = cv2.normalize(prev_hist, prev_hist)
          hist_dist = cv2.compareHist(hist_norm, prev_hist_norm, cv2.HISTCMP_CHISQR)
          if hist_dist > threshold:
              scene_change_count += 1

      # Update the previous frame and the frame counter
      prev_frame = resized
      frame_count += 1

      # Press 'q' to quit
      if cv2.waitKey(int(1000/fps)) & 0xFF == ord('q'):
          break

  # Determine if the scene has changed much
    percent_scene_change = scene_change_count / frame_count * 100
    if percent_scene_change > 1.5:
        print(i,'should be processed by videos to videos model')
        try:
            shutil.move(os.path.join(input_root,i), moving_root)
        except:
            print(i + ' not moved')
    else:
        print(i,'should be processed by photos to videos model')
        try:
            shutil.move(os.path.join(input_root,i), static_root)
        except:
            print(i + ' not moved')

帧率太高的话上一帧和下一帧区别不会太大

In [ ]:
input_root = '/content/drive/MyDrive/code_test/detect_scene/test_movie/'

In [ ]:
VideoCategories(input_root)

static.MP4 should be processed by videos to videos model
moving.MP4 should be processed by photos to videos model
static2.mp4 should be processed by photos to videos model
moving2.MOV should be processed by photos to videos model
moving3.MOV should be processed by videos to videos model
